In [ ]:
!git clone https://github.com/ctoshio/NER_Quechua.git

Cloning into 'NER_Quechua'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [ ]:
!git clone https://github.com/FrankCP67/NER-quechua-webapp.git

Cloning into 'NER-quechua-webapp'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 140 (delta 53), reused 108 (delta 23), pack-reused 0
Receiving objects: 100% (140/140), 576.79 KiB | 2.28 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
import pandas as pd
data = pd.read_csv("/content/NER-quechua-webapp/input/ner_dataset.csv")

In [ ]:
data = data.fillna(method = "ffill")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [ ]:
!pip install simpletransformers

In [ ]:
label = data["labels"].unique().tolist()
label

['B-ORG', 'O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-ORG']

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

In [ ]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', "/content/drive/MyDrive/proyecto/ner/QuechuaWordPiece" ,labels=label, args =args,use_cuda=True)

Some weights of the model checkpoint at /content/drive/MyDrive/proyecto/ner/QuechuaWordPiece were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassific

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:775: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


Running Epoch 1 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

(770, 0.6836036331467814)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/54 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 1.0516767369376288,
 'f1_score': 0.49536707056307916,
 'precision': 0.4978510028653295,
 'recall': 0.4929078014184397}

In [ ]:
prediction, model_output = model.predict(["Titiqaqa qucha, Buliwya"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Titiqaqa': 'B-LOC'}, {'qucha,': 'I-LOC'}, {'Buliwya': 'B-LOC'}]]

In [ ]:
prediction, model_output = model.predict(["Piruw"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Piruw': 'I-PER'}]]

In [ ]:
prediction, model_output = model.predict(["Imaynataq riyman Antabamba llaqtaman?"]) #¿Cómo llego a Antabamba? 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Imaynataq': 'B-LOC'},
  {'riyman': 'O'},
  {'Antabamba': 'O'},
  {'llaqtaman?': 'O'}]]